# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [4]:
llm_model = "deepseek-ai/DeepSeek-V3"

In [5]:
# !pip install pandas

In [37]:
import pandas as pd
df = pd.read_csv('Data.csv')

## LLMChain

In [10]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [11]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [12]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [14]:
chain = prompt | llm

In [16]:
product = "Queen Size Sheet Set"
chain.invoke(product)

AIMessage(content='A descriptive and appealing name for a company that specializes in Queen Size Sheet Sets could emphasize comfort, quality, and elegance. Here are some suggestions:\n\n1. **Royal Rest Linens**  \n2. **Queenly Comfort Co.**  \n3. **Slumber Luxe**  \n4. **Regal Threads**  \n5. **Dream Haven Sheets**  \n6. **Velvet Embrace Bedding**  \n7. **Majestic Sleep Co.**  \n8. **Elysian Linens**  \n9. **Crown Comfort Sheets**  \n10. **Serene Monarch Bedding**  \n\nThese names convey a sense of luxury and relaxation, which aligns well with premium bedding products. Choosing a name that resonates with your target audience and reflects your brand’s values is key!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 158, 'prompt_tokens': 20, 'total_tokens': 178, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='ru

## SimpleSequentialChain

In [17]:
from langchain.chains import SimpleSequentialChain

In [18]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = first_prompt | llm

In [23]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = second_prompt | llm

In [24]:
# 连接多个链
combined_chain = chain_one | chain_two

In [25]:
combined_chain.invoke(product)

AIMessage(content='"Luxurious queen-size sheet sets designed for ultimate comfort, elegance, and serene, restful nights."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 376, 'total_tokens': 398, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-8adb68fa-ef31-4b36-b280-09b50d4c783c-0', usage_metadata={'input_tokens': 376, 'output_tokens': 22, 'total_tokens': 398, 'input_token_details': {}, 'output_token_details': {}})

## SequentialChain

In [26]:
from langchain.chains import SequentialChain

In [45]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = (
    {"Review": RunnablePassthrough()}  # 传递输入
    | first_prompt                    # 应用 prompt
    | llm                             # 调用 LLM
    | {"English_Review": lambda x: x}  # 将输出存储到 "English_Review"
)


In [46]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = (
    {"English_Review": RunnablePassthrough()}  # 传递输入
    | second_prompt                    # 应用 prompt
    | llm                             # 调用 LLM
    | {"summary": lambda x: x}  # 将输出存储到 "English_Review"
)

In [47]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = (
    {"Review": RunnablePassthrough()}  # 传递输入
    | third_prompt                    # 应用 prompt
    | llm                             # 调用 LLM
    | {"language": lambda x: x}  # 将输出存储到 "English_Review"
)

In [48]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = (
    {"summary": RunnablePassthrough(), "language": RunnablePassthrough()}  # 传递输入
    | fourth_prompt                    # 应用 prompt
    | llm                             # 调用 LLM
    | {"followup_message": lambda x: x}  # 将输出存储到 "followup_message"
)

In [51]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = (
    {"Review": RunnablePassthrough()}  # 传递输入
    | RunnableParallel(
        {
            "English_Review": chain_one,
            "summary": chain_two,
            "language": chain_three,
        }
    )
    | RunnableParallel(
        {
            "followup_message": chain_four,
        }
    )
)

In [54]:
from langchain_core.tracers import ConsoleCallbackHandler
df = pd.read_csv('Data.csv')
review = df.Review[5]
response = overall_chain.invoke(review, config={"callbacks": [ConsoleCallbackHandler()]})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<Review>] Entering Chain run with input:
{
  "input": "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<Review> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<Review> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "Je tro

In [56]:
print(response)
type(response)

{'followup_message': {'followup_message': AIMessage(content="**Follow-up Response in French:**\n\nMerci pour votre commentaire détaillé. Nous sommes désolés d'apprendre que votre expérience n'a pas été à la hauteur de vos attentes. La qualité de nos produits est une priorité absolue, et il est inquiétant d'entendre que le goût et la mousse ne correspondaient pas à ce que vous avez l'habitude d'acheter en magasin. Nous prenons très au sérieux vos soupçons concernant un lot ancien ou contrefait. Afin de résoudre ce problème, nous vous invitons à nous fournir des détails supplémentaires sur le produit, tels que le numéro de lot et le lieu d'achat, afin que nous puissions enquêter et prendre les mesures appropriées. Nous tenons à vous assurer que nous ferons tout notre possible pour garantir votre satisfaction et maintenir la qualité de nos produits. N'hésitez pas à nous contacter directement pour toute assistance supplémentaire.  \n\nCordialement,  \n[Votre nom/Équipe de support client]",

dict

## Router Chain

In [71]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [72]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [73]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [74]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [75]:
from langchain_core.output_parsers import StrOutputParser
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = prompt | llm | StrOutputParser()
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [76]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = default_prompt | llm | StrOutputParser()

In [77]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [85]:
import json
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由链
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=StrOutputParser(),
)

def route_input(input_data):
    # 调用 LLM 进行路由
    response = llm.invoke(router_prompt.format(input=input_data))
    # 从 AIMessage 中提取 content
    response_content = response.content
    # 解析 JSON 输出
    try:
        # 去除 Markdown 代码块的标记（如 ```json 和 ```）
        response_content = response_content.strip("```json").strip("```")
        result = json.loads(response_content)
        return result
    except json.JSONDecodeError:
        # 如果解析失败，返回默认值
        return {"destination": "DEFAULT", "next_inputs": input_data}

# 组合链
def multi_prompt_chain(input_data):
    routing_result = route_input(input_data)
    destination = routing_result["destination"]
    next_inputs = routing_result["next_inputs"]
    print(f"destination:{destination}, \nnext_inputs:{next_inputs}")
    
    if destination in destination_chains:
        return destination_chains[destination].invoke(next_inputs)
    else:
        return default_chain.invoke(next_inputs)

In [86]:
multi_prompt_chain("What is black body radiation?")

destination:physics, 
next_inputs:What is black body radiation?


"Black body radiation is the electromagnetic radiation emitted by an idealized object called a black body, which absorbs all incident radiation and re-emits it based solely on its temperature. A black body is a perfect absorber and emitter of energy, meaning it doesn’t reflect or transmit any radiation.\n\nThe spectrum of black body radiation depends only on the object's temperature and follows Planck's law. As the temperature increases, the peak wavelength of the emitted radiation shifts to shorter wavelengths (Wien's displacement law), and the total energy radiated increases (Stefan-Boltzmann law). This phenomenon explains the color and intensity of light emitted by hot objects, like stars or heated metals."

In [87]:
multi_prompt_chain("what is 2 + 2")

destination:math, 
next_inputs:what is 2 + 2


'To solve the question "What is 2 + 2?", let\'s break it down into its component parts and then put them together to arrive at the answer.\n\n### Step 1: Understand the numbers\n- The numbers involved are **2** and **2**.\n\n### Step 2: Understand the operation\n- The operation is **addition (+)**. Addition means combining two or more numbers to find their total.\n\n### Step 3: Perform the addition\n- Add the two numbers:  \n  \\( 2 + 2 = 4 \\)\n\n### Step 4: Final Answer\nThe result of \\( 2 + 2 \\) is **4**.'

In [88]:
multi_prompt_chain("Why does every cell in our body contain DNA?")

destination:DEFAULT, 
next_inputs:Why does every cell in our body contain DNA?


"Every cell in our body contains DNA because DNA carries the essential genetic instructions necessary for the cell's growth, development, functioning, and reproduction. Here’s why DNA is present in every cell:\n\n1. **Genetic Blueprint**: DNA contains the genes that encode the information needed to build and maintain the organism. These genes provide the instructions for synthesizing proteins, which perform most of the functions in the cell.\n\n2. **Cellular Function**: Each cell, regardless of its type or function, requires specific proteins to carry out its tasks. DNA provides the code for these proteins, ensuring that each cell can perform its role effectively.\n\n3. **Cell Division and Reproduction**: When cells divide (through mitosis or meiosis), they need to replicate their DNA to pass on the genetic information to the daughter cells. This ensures that each new cell has the same genetic instructions as the parent cell.\n\n4. **Regulation of Cellular Activities**: DNA also contai